In [1]:
!pip install langgraph langchain-core langchain-openai --quiet
print("✅ Libraries installed.")

import os
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, BaseMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from typing import TypedDict, Annotated, Sequence, Literal
import operator
import uuid

try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    print("✅ OpenAI API Key set successfully.")
except (ImportError, userdata.SecretNotFoundError):
    print("⚠️ API Key not found. Please set it in Colab Secrets.")

✅ Libraries installed.
✅ OpenAI API Key set successfully.


In [2]:
@tool
def get_leave_balance(employee_id: str) -> dict:
    """Gets the available leave balance for a given employee ID."""
    print(f"--- TOOL: Checking leave balance for {employee_id} ---")
    return {"vacation": 10, "sick": 5}

@tool
def get_employee_salary(employee_id: str) -> str:
    """
    Retrieves the annual salary for a given employee ID.
    This is a sensitive tool and its output should be handled with care.
    """
    print(f"--- TOOL: Accessing sensitive salary data for {employee_id} ---")
    return "The annual salary for emp_123 is $80,000."

In [3]:
class HRAgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    employee_name: str
    user_role: str
    conversation_summary: str
    pii_detected: bool
    escalation_reason: str
    escalation_ticket_id: str
    remaining_steps: int

In [4]:
def summarization_node(state: HRAgentState) -> dict:
    print("\n--- 🧠 PRE-MODEL HOOK: Summarizer ---")
    num_messages = len(state['messages'])
    if num_messages > 4:
        print(f"🔎 Conversation has {num_messages} messages. Summarizing...")
        summary = f"The conversation with {state['employee_name']} has covered leave balances and other HR topics."
        summary_msg = SystemMessage(content=f"[Conversation Summary: {summary}]")
        recent_msgs = [m for m in state['messages'] if not isinstance(m, SystemMessage)][-3:]
        new_messages = [summary_msg] + recent_msgs
        print(f"✅ Context window reduced from {num_messages} to {len(new_messages)} messages.")
        return {"messages": new_messages, "conversation_summary": summary}
    print(f"✅ Conversation has {num_messages} messages. No summarization needed.")
    return {}

def dynamic_prompt_node(state: HRAgentState) -> dict:
    print("--- 🧠 PRE-MODEL HOOK: Dynamic Prompter ---")
    prompt_parts = [
        f"You are a helpful HR assistant chatting with {state['employee_name']}, who holds the role of '{state['user_role']}'."
    ]
    if state['user_role'] == 'manager':
        prompt_parts.append("As a manager, you are authorized to use tools for approvals and team data.")
    else:
        prompt_parts.append("As an employee, you can use tools to check your own personal data.")

    system_prompt = "\n".join(prompt_parts)
    print(f"✅ Dynamic system prompt injected.")
    return {"messages": [SystemMessage(content=system_prompt)]}

In [5]:
def agent_node(state: HRAgentState, agent_runnable) -> dict:
    print("--- 🤖 AGENT EXECUTING ---")
    # The agent runnable is now invoked with the entire state. It will find the
    # 'messages' key, which now includes the dynamic system prompt.
    result = agent_runnable.invoke(state)
    print("--- ✅ AGENT FINISHED ---")
    return result

In [6]:
def guardrail_and_route(state: HRAgentState) -> Literal["escalate", "__end__"]:
    print("--- 🚦 POST-MODEL HOOK & ROUTER: Guardrail ---")
    last_message = state['messages'][-1]
    if not isinstance(last_message, AIMessage):
        return "__end__"

    content = last_message.content.lower()
    if "salary" in content or "$" in content:
        print("🚨 PII DETECTED! Routing to escalation path.")
        return "escalate"

    print("✅ No PII detected. Ending the turn.")
    return "__end__"

In [7]:
def escalation_node(state: HRAgentState) -> dict:
    print("--- 🔥 ESCALATION PATH ---")
    ticket_id = f"HR-INCIDENT-{uuid.uuid4().hex[:8].upper()}"
    reason = "PII Leak Attempt"
    print(f"🔥 Escalating to HR. Reason: {reason}. Ticket ID: {ticket_id}")

    last_message = state['messages'][-1]
    redacted_msg = AIMessage(content="⚠️ I cannot disclose sensitive PII. This event has been flagged for manual review.", id=last_message.id)

    return {"messages": [redacted_msg], "pii_detected": True, "escalation_reason": reason, "escalation_ticket_id": ticket_id}

print("✅ All agent components defined.")

✅ All agent components defined.


In [8]:
def main():
    print("\n--- Lab 2.5: Fully Integrated System (Flawless Version) ---")
    model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    tools = [get_leave_balance, get_employee_salary]

    # The prompt is now much simpler. It only needs to know where to find the messages.
    # The dynamic system prompt will already be part of the `messages` list.
    prompt = ChatPromptTemplate.from_messages([
        MessagesPlaceholder(variable_name="messages"),
    ])

    # We use the standard create_react_agent constructor.
    agent_runnable = create_react_agent(model, tools, prompt=prompt)

    # --- Assemble the Graph ---
    workflow = StateGraph(HRAgentState)
    workflow.add_node("summarizer", summarization_node)
    workflow.add_node("dynamic_prompter", dynamic_prompt_node)
    workflow.add_node("agent", lambda state: agent_node(state, agent_runnable))
    workflow.add_node("escalation", escalation_node)

    workflow.set_entry_point("summarizer")
    workflow.add_edge("summarizer", "dynamic_prompter")
    workflow.add_edge("dynamic_prompter", "agent")
    workflow.add_conditional_edges(
        "agent",
        guardrail_and_route,
        {
            "escalate": "escalation",
            "__end__": END
        }
    )
    workflow.add_edge("escalation", END)

    app = workflow.compile()

    # --- Test Case: PII Query (Triggers Guardrail/Escalation) ---
    print("\n\n" + "="*60)
    print("🚀 TEST CASE: PII Query (Triggers Guardrail/Escalation)")
    print("="*60)

    # The initial state must contain all keys defined in HRAgentState
    pii_convo = [HumanMessage(content="What is the annual salary for employee emp_123?")]
    state_pii = {
        "messages": pii_convo,
        "employee_name": "Charlie",
        "user_role": "manager",
        "remaining_steps": 5,
        "conversation_summary": "",
        "pii_detected": False,
        "escalation_reason": "",
        "escalation_ticket_id": "",
    }
    result_pii = app.invoke(state_pii)

    print("\n" + "="*50)
    print("--- FINAL INTEGRATED SYSTEM OUTPUT (PII TEST) ---")
    print(f"PII Detected and Blocked: {result_pii.get('pii_detected', False)}")
    print(f"Escalation Ticket ID: {result_pii.get('escalation_ticket_id', 'N/A')}")
    print("\n--- Final Agent Response (After all hooks) ---")
    print(result_pii['messages'][-1].content)
    print("="*50)

if __name__ == "__main__":
    main()


--- Lab 2.5: Fully Integrated System (Flawless Version) ---


🚀 TEST CASE: PII Query (Triggers Guardrail/Escalation)

--- 🧠 PRE-MODEL HOOK: Summarizer ---
✅ Conversation has 1 messages. No summarization needed.
--- 🧠 PRE-MODEL HOOK: Dynamic Prompter ---
✅ Dynamic system prompt injected.
--- 🤖 AGENT EXECUTING ---
--- TOOL: Accessing sensitive salary data for emp_123 ---
--- ✅ AGENT FINISHED ---
--- 🚦 POST-MODEL HOOK & ROUTER: Guardrail ---
🚨 PII DETECTED! Routing to escalation path.
--- 🔥 ESCALATION PATH ---
🔥 Escalating to HR. Reason: PII Leak Attempt. Ticket ID: HR-INCIDENT-C04AB050

--- FINAL INTEGRATED SYSTEM OUTPUT (PII TEST) ---
PII Detected and Blocked: True
Escalation Ticket ID: HR-INCIDENT-C04AB050

--- Final Agent Response (After all hooks) ---
⚠️ I cannot disclose sensitive PII. This event has been flagged for manual review.
